# Linux File System:-

## import libraries

In [1]:
# Importing the datetime class from the datetime module
from datetime import datetime

# Importing the random module and aliasing it as 'rd'
import random as rd

# Importing the math module and aliasing it as 'm'
import math as m

# Importing the uuid module for working with UUIDs
import uuid


## create root => directory => file

In [71]:
class root:
    def __init__(self, root_label, size):
        # Initializing instance variables
        self.parent = None 
        self.root_label = root_label  # define root label
        self.identifier = str(uuid.uuid4())  # Generating a unique identifier for the root
        self.size = size  # Size of the root in bytes
        self.used_space = 0  # Initializing used space to 0
        self.free_space = self.size - self.used_space  # Calculating free space
        self.n_blocks = m.floor(self.size / 4)  # Calculating the number of blocks based on size (block size is 4 bytes)
        self.children = [Directory("users")]  # Creating a list with a single Directory object
        self.children[0].parent = self  # Setting the parent of the Directory object to this root
        self.children[0].location = f"{self.root_label}:\\"
        # Dictionary to store user accounts within the root
        self.user_accounts = {}
        # Creating an initial user account for 'Administrator' with default password and role
        self.create_user("Administrator", str(12345), "Admin")
    
    def authenticate(self, username, password):
        # Authenticating users based on username and password
        if username in self.user_accounts and self.user_accounts[username][0] == password:
            return True  # Authentication successful
        return False  # Authentication failed

    def create_user(self, username, password, role):
        # Creating a new user account
        if username not in self.user_accounts:
            self.user_accounts[username] = [password, role]  # Storing user details in a dictionary
            print(f"User '{username}' created successfully with role: {role}")
            self.children[0].add(Directory(f"{username}"))  # Adding a directory for the user
        else:
            print("Username already exists. Please choose another username.")

    def modify_user(self, username, new_username, new_password):
        # Modifying an existing user's details
        if username in self.user_accounts:
            # Updating user details with new username and/or password
            self.user_accounts[new_username] = [new_password, self.user_accounts.pop(username)[1]]
            print(f"User '{username}' modified successfully to '{new_username}'")
        else:
            print("User not found. Cannot modify.")
            
    def display_root_content(self):
    # Display the content of the root
        for child in self.children:
            child.display_content()

In [72]:
class File: 
    def __init__(self, file_name, location, Type, protection, content=""):
        # Initializing file attributes
        self.name = file_name
        self.content = content
        self.size = len(str(content))  # Calculate size based on content 
        self.location = location  # Location within the root
        self.protection = protection
        self.identifier = rd.randint(0, 99999)  # Generating a random identifier
        self.type = "." + Type
        self.blocks = m.ceil(self.size / 4)  # Calculating the number of blocks based on size
        self.creation_time = datetime.now()
        self.modification_time = datetime.now()
        self.access_time = datetime.now()
        self.parent = None  # Parent directory (not used in methods)
        
        # we didn't define self.children for file node as file node always will be the leaf node

    def read_content(self):
        # Read and update access time
        self.access_time = datetime.now()
        return self.content

    def write_content(self, new_content):
        # Write new content, update attributes, and access time
        self.access_time = datetime.now()
        self.content = new_content
        self.modification_time = datetime.now()
        self.size = len(new_content)
        self.blocks = m.ceil(self.size / 4)

    def change_protection(self, new_protection, account_type):
        # Change protection if account type is "Admin", update access and modification times
        if account_type == "Admin":
            self.access_time = datetime.now()
            self.protection = new_protection
            self.modification_time = datetime.now()
        else:
            print("Unable to change protection. Please log in as an administrator.")

    def Rename(self, new_name):
        # Rename file, update attributes, and modification time
        self.access_time = datetime.now()
        self.name = new_name
        self.modification_time = datetime.now()

    def get_information(self):
        # Display file information with formatted sizes and timestamps
        def format_size(size_in_bytes):
            units = ['B', 'KB', 'MB', 'GB']
            unit_index = 0

            while size_in_bytes >= 1024 and unit_index < len(units) - 1:
                size_in_bytes /= 1024.0
                unit_index += 1

            return f"{size_in_bytes:.2f} {units[unit_index]}"

        print(
            "Name: ", self.name,
            "\nIdentifier: ", self.identifier,
            "\nLocation: ", self.location,
            "\nType: ", self.type,
            "\nSize: ", format_size(self.size),
            "\nBlocks: ", self.blocks,
            "\nProtection: ", self.protection,
            "\nCreation Time: ", self.creation_time.strftime("%Y-%m-%d %H:%M:%S"),
            "\nModification Time: ", self.modification_time.strftime("%Y-%m-%d %H:%M:%S"),
            "\nAccess Time: ", self.access_time.strftime("%Y-%m-%d %H:%M:%S")
        )

In [73]:
class Directory(File):
    """
    Explanation:
    The Directory class inherits from the File class and manages directories.
    """   
    def __init__(self, directory_name):
        # Initialize Directory attributes
        super().__init__(directory_name, "", "Folder", protection="rw+")
        self.permissions = {
            "Admin": "rw",  # Admin has read and write access by default
            "Guest": "r"    # Guest has read-only access by default
        }
        self.children = []  # To hold files or subdirectories
        self.parent = None  # Parent directory 

    def change_directory_permissions(self, new_permissions, account_type):
        # Change directory permissions if account type is "Admin"
        if account_type in self.permissions and account_type == "Admin":
            self.permissions[account_type] = new_permissions
            print(f"Directory permissions changed to '{new_permissions}' successfully.")
        else:
            print("Unable to change directory permissions. Login as administrator required.")
    
    def add(self, child):
        # Add a child (file or subdirectory) to the directory
        self.children.append(child)
        self.Calculation()  # Recalculate sizes and blocks
        child.parent = self
        self.Adjust_location(child)  # Adjust location of the child within the directory
     

    def copy_file(self, file, destination_directory):
        # Create a copy of the file and add it to the destination directory
        copied_file = File(file.name,  destination_directory.name, file.type, file.protection, 4)
        # Copy file attributes
        copied_file.content = file.content
        copied_file.size = file.size
        copied_file.blocks = file.blocks
        copied_file.creation_time = file.creation_time
        copied_file.modification_time = file.modification_time
        copied_file.access_time = file.access_time
        destination_directory.add(copied_file)  # Add copied file to the destination directory
        destination_directory.Calculation()  # Recalculate sizes and blocks
        copied_file.parent = destination_directory  # Set the copied file's parent as the destination directory
        self.Adjust_location(copied_file)  # Adjust the copied file's location
        
    def delete(self, file_name):
        # Delete a file with the given name from the directory
        for child in self.children:
            if isinstance(child, File) and child.name == file_name:
                self.children.remove(child)
                break
        self.Calculation()  # Recalculate sizes and blocks after deletion
        
    def move(self, file_name, destination_directory):
        # Move a file with the given name to the destination directory
        for child in self.children:
            if isinstance(child, File) and child.name == file_name:
                destination_directory.add(child)  # Add the file to the destination directory
                child.parent = destination_directory  # Update the file's parent
                self.children.remove(child)  # Remove the file from the current directory
                self.Adjust_location(child)  # Adjust the file's location
                break
        self.Calculation()  # Recalculate sizes and blocks after moving
        
    def search(self, target_name):
        # Search for a file or directory with the given target name
        founded = []
        
        def dfs(node, target_name):
            if node is None:
                return "couldn't find what you are looking for"
            if node.name == target_name:
                founded.append(node)
            if isinstance(node, Directory):
                for child in node.children:
                    dfs(child, target_name)
        
        dfs(self, target_name)
        
        return founded

    def Calculation(self, parent=None):
        # Recalculate sizes and blocks within the directory structure
        parent = self if parent is None else parent
        if hasattr(parent, 'root_label'):  # Check if the parent has the attribute 'root_label'
            # Perform calculations specific to the 'root' class here
            parent.used_space = sum(child.size for child in parent.children)
            parent.free_space = parent.size - parent.used_space
        else:
            # Perform calculations for other types of parent directories
            parent.size = sum(child.size for child in parent.children)
            parent.blocks = sum(child.blocks for child in parent.children)
        if parent.parent:
            self.Calculation(parent.parent)  # Recursive call to calculate parent directory sizes

    
    def Adjust_location(self, child):
        # Adjust the location of a child file or directory within the directory structure
        child.location = child.parent.location + f"{child.parent.name}\\"
        if isinstance(child, Directory) and child.children:
            for kid in child.children:
                self.Adjust_location(kid)
        else:
            return
            
    def display_content(self, indent=0):
        # Display the content of the directory in a tree-like structure
        for child in self.children:
            if isinstance(child, Directory):
                print(" " * indent + f"Directory: {child.name} - Location: {child.location}")
                child.display_content(indent + 4)  # Recursive call for subdirectories
            elif isinstance(child, File):
                print(" " * indent + f"File: {child.name} ({child.type}) - Location: {child.location}")

## test the code

In [74]:
# creating a file with large content to test
content = """he vast expanse of the cosmos, where galaxies twirl in a cosmic dance and stars flicker 
like distant candles, a tapestry of celestial wonders unfolds. Nebulas, those ethereal clouds of gas and dust, 
paint the cosmic canvas with hues of emerald and ruby, as if nature itself were an artist wielding the brush of creation. 
In the quiet corners of the universe, black holes lurk like cosmic phantoms, 
their gravitational embrace bending the very fabric of space and time.
As the cosmic clock ticks away eons, planets orbit their parent stars, 
each with its own unique story etched in the geological tapestry of its surface.
Moons, silent witnesses to the drama unfolding in the cosmic theater, cast their soft glow upon the landscapes below.
In the dance of cosmic forces, supernovae unleash torrents of energy, scattering the elements forged in their
fiery hearts across the cosmic stage, contributing to the grand cycle of stellar birth and death.
Amidst this cosmic ballet, life, a precious rarity, emerges on a pale blue dot suspended in the sunbeam of a star. 
On Earth, oceans teem with diverse life forms, and lush forests breathe with the rhythm of existence. 
Creatures of every shape and size navigate the intricate web of ecosystems, each playing a role in the grand symphony of life. 
Human intellect, a spark of consciousness, seeks to unravel the mysteries of the cosmos, 
reaching out to the stars with a thirst for knowledge that transcends the boundaries of our home planet.
Through the corridors of time, civilizations rise and fall, leaving behind echoes of their existence in the annals of history. 
The written word, a testament to human creativity, weaves tales of triumph and tragedy, of love and loss, 
forming a mosaic of human experience. Art, in its myriad forms, captures the essence of emotion and thought,
preserving the collective soul of humanity across the ages.
In the boundless realms of imagination, dreams take flight like cosmic butterflies, 
fluttering through the corridors of possibility. Science and technology, the twin engines of human progress, propel us towards the stars,
as we strive to unlock the secrets of the universe and chart our course through the cosmic ocean. 
And so, the journey continues, an odyssey of discovery, 
an exploration of the infinite possibilities that await in the ever-expanding tapestry of existence."""

f1 = File("hello",":\\","txt","rw-",content)
f1.get_information()

Name:  hello 
Identifier:  31083 
Location:  :\ 
Type:  .txt 
Size:  2.32 KB 
Blocks:  595 
Protection:  rw- 
Creation Time:  2024-01-04 18:45:37 
Modification Time:  2024-01-04 18:45:37 
Access Time:  2024-01-04 18:45:37


In [75]:
# D1 have no content now
D1 = Directory("D1")
D1.get_information()

Name:  D1 
Identifier:  86916 
Location:   
Type:  .Folder 
Size:  0.00 B 
Blocks:  0 
Protection:  rw+ 
Creation Time:  2024-01-04 18:45:38 
Modification Time:  2024-01-04 18:45:38 
Access Time:  2024-01-04 18:45:38


In [76]:
D2 = Directory("D2")
D3 = Directory("D3")
Mina = File("Mina",":\\","txt","rw-","Miano pleasure to meet you.")
Luna = File("Luna",":\\","txt","rw-","nice to meet you this is me ?")

D1.add(D2)#add directory 2 to the directory 1
D2.add(D3)# add directory 3 to the directory 2
D2.add(Mina)# add file to the directory 2
D1.add(Luna)# add file to the directory 1

print()
"""
Explaination :
File 1 : len("Miano pleasure to meet you.") : 27 => 27/ size of block (4) =>6.75 (7 blocks)
File 2 : len("nice to meet you this is me ?") : 29 => 29/4 => 7.25 (8 blocks)

Directory Inherited the attributes of file so the size used in directory =>  27 + 29 = 56 byte
the total blocks used in directory  => 7 + 8 = 15 block
"""
D1.get_information()


Name:  D1 
Identifier:  86916 
Location:   
Type:  .Folder 
Size:  56.00 B 
Blocks:  15 
Protection:  rw+ 
Creation Time:  2024-01-04 18:45:38 
Modification Time:  2024-01-04 18:45:38 
Access Time:  2024-01-04 18:45:38


In [77]:
D1.search("Mina")#will print objects that matches the same name and append it to list

In [78]:
# since search function return list containing objects that has the same name
# we access this list by indices to show object 
D1.search("Mina")[0].content 

'Miano pleasure to meet you.'

In [79]:
D1.display_content() #display tree of parent

Directory: D2 - Location: D1\
    Directory: D3 - Location: D1\D2\
    File: Mina (.txt) - Location: D1\D2\
File: Luna (.txt) - Location: D1\


In [80]:
#copy Mina file and change the destination of the file to the directory3
D1.copy_file(D1.search("Mina")[0],D1.search("D3")[0])
D1.display_content()

Directory: D2 - Location: D1\
    Directory: D3 - Location: D1\D2\
        File: Mina (..txt) - Location: D1\D2\D3\
    File: Mina (.txt) - Location: D1\D2\
File: Luna (.txt) - Location: D1\


In [81]:
# move lina into D2 
# D1 conatins D2
# D2 contain luna and mina
D1.move("Luna",D1.search("D2")[0])
D1.display_content()

Directory: D2 - Location: D1\
    Directory: D3 - Location: D1\D2\
        File: Mina (..txt) - Location: D1\D2\D3\
    File: Mina (.txt) - Location: D1\D2\
    File: Luna (.txt) - Location: D1\D2\


In [82]:
D2.display_content()

Directory: D3 - Location: D1\D2\
    File: Mina (..txt) - Location: D1\D2\D3\
File: Mina (.txt) - Location: D1\D2\
File: Luna (.txt) - Location: D1\D2\


In [83]:
# add the large file (hello) in D1
D1.add(f1)

In [84]:
# f1 is added
D1.display_content()

Directory: D2 - Location: D1\
    Directory: D3 - Location: D1\D2\
        File: Mina (..txt) - Location: D1\D2\D3\
    File: Mina (.txt) - Location: D1\D2\
    File: Luna (.txt) - Location: D1\D2\
File: hello (.txt) - Location: D1\


In [85]:
# create root file
root = root("Root", 10000)

User 'Administrator' created successfully with role: Admin


In [86]:
# create user file will be the first account 
user = root.children[0].search("Administrator")[0]

In [87]:
# now there is no contents in user directory
user.display_content()

In [17]:
# root have Administrator file account
root.display_root_content()

Directory: Administrator - Location: Root:\users\


In [88]:
# add content in user
user.add(D1)
user.add(f1)

In [89]:
# add D1(Contains(D2=>luna,mina,hello)) and hello
user.display_content()

Directory: D1 - Location: Root:\users\Administrator\
    Directory: D2 - Location: Root:\users\Administrator\D1\
        Directory: D3 - Location: Root:\users\Administrator\D1\D2\
            File: Mina (..txt) - Location: Root:\users\Administrator\D1\D2\D3\
        File: Mina (.txt) - Location: Root:\users\Administrator\D1\D2\
        File: Luna (.txt) - Location: Root:\users\Administrator\D1\D2\
    File: hello (.txt) - Location: Root:\users\Administrator\
File: hello (.txt) - Location: Root:\users\Administrator\


In [90]:
# delete file now conatian D1 only
user.delete("hello")
user.display_content()

Directory: D1 - Location: Root:\users\Administrator\
    Directory: D2 - Location: Root:\users\Administrator\D1\
        Directory: D3 - Location: Root:\users\Administrator\D1\D2\
            File: Mina (..txt) - Location: Root:\users\Administrator\D1\D2\D3\
        File: Mina (.txt) - Location: Root:\users\Administrator\D1\D2\
        File: Luna (.txt) - Location: Root:\users\Administrator\D1\D2\
    File: hello (.txt) - Location: Root:\users\Administrator\


In [91]:
# check size in bytes
print(root.size,"\n",root.used_space,"\n", root.free_space)

10000 
 2460 
 7540


In [92]:
# delete anther file from D1
D1.delete("hello")
user.display_content()

Directory: D1 - Location: Root:\users\Administrator\
    Directory: D2 - Location: Root:\users\Administrator\D1\
        Directory: D3 - Location: Root:\users\Administrator\D1\D2\
            File: Mina (..txt) - Location: Root:\users\Administrator\D1\D2\D3\
        File: Mina (.txt) - Location: Root:\users\Administrator\D1\D2\
        File: Luna (.txt) - Location: Root:\users\Administrator\D1\D2\


In [93]:
# check size in bytes => used space in small now as hello file was large
print(root.size,"\n",root.used_space,"\n", root.free_space)

10000 
 83 
 9917
